<a href="https://colab.research.google.com/github/jobdiogenes/data-science-recipes/blob/master/gists/data_science_gist_02_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Índice](https://github.com/jobdiogenes/data-science-recipes/blob/master/gists/index.md)
# Série de Receitas para Ciência de Dados
## Episódio 2 - Repositórios: extraindo dados do Fishbase com python usando REST API  
### Introdução
>**Mas que diabos é isso?** Geralmente ao ver siglas de áreas que não conhecemos ficamos assustados, ou mesmo nas áreas que atuamos sempre aparece uma determinada sigla que não conhecemos ou que tem um signifcado diferente em especilidades diferentes. Então, primeiramente não se assuste, **API** é apenas mais uma sigla. 

>**API** vem do inglês de Application Program Interface, ou seja Interface de Programação de Aplicativos. Que nada mais é do que um conjunto de regras definidas para se comunicar com um Aplicativo (software). Na biologia pense nisso como um taxonomista e nos métodos padronizados para descrever e classificar uma espécie, gênero, ou família por exemplo. Logo a frente você perceber muita semelhança da organização de uma **API** do tipo **REST** (_Representational State Transfer_)  com uma hierarquia taxonômica e detalhamentos.

Pesquisadores e cientistas são ávidos por base da dados (repositórios), para obter informações para suas pesquisas, que podem ser uma uma simples confirmação de nome científico, usar banco de dados genéticos para comparar com dados genéticos seus, ou extrair massas de dados para fazer análises sobre elas, usando critérios espaciais como na ecologia. Não, não se preocupe, não vamos fazer uma análise espacial multivariada agora, nem aplicar aprendizado de máquina  para identificar padrões de expressão e comunicação automaticamente. Aqui é um tijolo de cada vez.  

No entanto, quando começamos a fazer consultas à essas bases,  hoje em dia, o fazemos através de navegadores, indo até o endereço de internet (URL) da mesma, e usamos sua interface para realizar as consultas. É claro esse modo de pesquisa nos ajuda, porém começa a ficar pouco prático quando quantidade de consultas vai crescendo e a diversidade de informações vai expandindo. E o pior, para coletar os dados de resultados temos que ficar selecionado, copiando e colando, que além de cansativo podemos cometer erros. Fazer isso uma ou 2 vezes, é tranquilo, mas conforme vai aumentando o esforço cresce muito, nos causa cansaço visual e até Lesão por Esforço Repetitivo, e toma muito tempo. Um pesquisador agindo assim pode ter de reduzir muito a amplitude de dados pesquisados, pois vai consumir muito tempo e poderá ainda cometer erros ou pior ter gasto muito tempo para obter algo que não atendeu as necessidades. 

Para um pesquisador com prazos, pode ser e emocionalmente muito desgantante ter de gastar muito tempo para obter e analisar dados de forma manual, pelo tempo que consome e a pressão para não cometer erros. E se isso ainda não atender as necessidades do plano de pesquisa ou insuficientes, pode se tornar algo extremamente frustante. 

>**PS** é também possível extrair dado de forma automática de repositórios que não possuem API e só disponibilizam uma interface WEB. No entanto, isso é um pouco mais complexo e não iremos tratar disso por enquanto. Outra questão de não fazer isso, é que alguns repositórios não oferecem API pois não permitem extração de dados de forma automatizada, o que é uma pena. 

### Meta
Por isso a meta desse episódio é tornar familiar aos pesquisadores, os conceitos e a técnicas para extração de dados de repositórios usando API, para que assim possam fazer suas próprias pesquisas de forma automática, escrevendo suas próprias rotinas, ou quando avaliar uma rotina desenvolvida por outros, ser capaz de entender, conhecer seus limites ou mesmo identificar erros. Mesmo que não se torne um expert, que espero que se tornem ao final, mas conhecer como isso é feito é fundamental para qualquer pesquisador, pois amplia sua capacidade de visualizar os dados e compreender as amostras extraídas por outrem.

### O que iremos ver ?
Bom, antes de mais nada, para realizar os exercícios desse episódio é importante já estar familiar com o conteúdo do [Episódio 1](https://gist.github.com/jobdiogenes/a8d65fcabc8f8a47473769a9ba970fe0). Então agora abra o Python 3, num dos meios sugeridos no episódio 1. E

Neste episódio temos os seguintes iténs:
1. [X] Descrição de como funcionam API REST de repositórios.
1. [X] Extraindo dados Fishbase usando a API REST.

> Como dito no primeiro episódio, os novos episódios serão menores.

### Item 1 - Descrição de uma API REST
Bom antes de falar sobre a API REST, em uma nova aba do navegador de internet vá a página de pesquisa [FishBase.org](www.fishbase.org), para você se familiarizar com o conteúdo oferecido pelo repositório. Como eu disse no primeiro episódio o primeiro passo é sempre conhecer o que oferecem os repositórios, para depois aos poucos criar sua estratégia de extração de dados. 

E lá faça uma pesquisa pela espécie:  _Serrasalmus marginatus_, no resultado da pesquisa retorna uma página rica em informações,
com foto da espécie, classificação, descrição, etc. A forma apresentada atende quem esta visualizando a página e é possível extrair informações copiando e colando e até as imagens.

Esse conteúdo que recebemos ao acessar uma página por um navegador usa o protocolo HTTP, que vemos sempre nos endereços no navegador e o coteúdo que retorna ao navegador é em geral o HTML, que por fim é processado e apresentado ao usuário. 

Por exemplo o conteúdo HTML
```html
<h1>Peixes predadores </h2>
<h2>Piranha</h2>
<p>Nome científico: <i>Serrasalmus marginatus</i></p>
```

O navegador processa e exibe como: 

> # Peixes predadores
>## Piranha
>Nome científico:  _Serrasalmuls mergisnatus_

O protocolo HTTP também usa os conceitos do REST, por exemplo quando digitamos:
```
https://en.wikipedia.org/wiki/Piranha
```

Estamos através do HTTP usando os conceitos do REST, onde após digitar o endereço o navegador (Firefox, Chrome, etc) envia um comando de solicitação de dados ao endereço de domínio destino (en.wikipedi.org) contendo os parâmetros **/wiki/Piranha**. 

No entanto conforme a WEB foi desenvolvendo o conteúdo do HTML foi crescendo incluindo uma série de recursos para mellhorar o visual, exibir imagens, sons e vídeos, lidar com a interface com o usuário.  Para ver essa complexidade, você pode clicar em algum lugar vazio desta página com o botão direito e pedir para ver o código fonte. Vai encontrar um conteúdo grande, que não lhe parecerá muito compreensível.

No entando se você esta apenas interessado em dados, não faz sentido receber um conteúdo tão grande complexo. As API REST, como outras APIS de começaram então a se desenvolver para atender a essa necessidade. Ao invés do HTML, as API REST se comunicam em geral com 3 tipos de formatos, são eles:

>**XML**
```XML
<fish>
   <genus>Serrasalmus</genre>
   <species>spilopleura</specie>
   <author>Kner, 1858</autor>
   <spec_code>11973</spec_code>
</fish>   
```

> **JSON**
```json
{ fish: {
      genus: "Serrasalmus",
      species: "spiplopleura",
      author: "Kner, 1858",
      "spec code": 11973
      }
}
```

>**YAML**
```yaml
fish:
genus: Serrasalmus
species: spilopleura
author: Kner, 1958
spec code: 11973
```

São mais compreensívos que o HTML, pois se quer apenas dados e lidar com o visual de uma página.
A maioria dos repositórios permitem em geral XML e JSON, e algumas o YAML, o Python oferece recursos, através de **bibliotecas**  para lidar com os três formatos. Mas em geral usamos mais o JSON, pois é menos verboso e mais rápido. O YAML, quando disponível é muito útil quando queremos ver o conteúdo do repositório para entender.

A comunicação em geral é feita pelo cliente (o seu programa) solicitando ao repositório informações através de parâmetros passados.
E após enviarmos esses parâmetros, tratamos as informações que retornam.  

Em geral temos temos de saber se os parêmetros que passamos estão corretos e retornam dados, ou se deu algum erro. Os códigos seguem o modelo  padrão HTTP com algumas modificações em alguns casos, mas no geral são os mesmos. Por exemplo para resposta sem erro o código é: **200**. 

A documentação da API do fish base pode ser encontrada aqui:  (https://fishbaseapi.readme.io)

Vamos ao primeiro exemplo: 








In [0]:
# Primeiro importamos a função get da biblioteca requests que é uma da bibliotecas 
# disponíveis para lidar com consultas a endereços de internet (URL)
# poderiamos importar toda a biblioteca escrevendo: import requests as r 
# onde o r seria um apelido e o uso da função get como outras funções disponíveis 
# seria feita como r.get() 
#
from requests import get
# definimos uma variável
h = "https://fishbase.ropensci.org/"
# vamos pesquisar por Serrasalmus marginatus
# maisculos e minusculos são desconsiderados
r = get(h+'/taxa?Genus=serrasalmus&Species=spilopleura')
# exibir o codigo do retorno
print('Codigo R:',r.status_code)
# exibindo o texto de conteúdo
print(r.text)
print(type(r.text))

# algo com rota (route) errada (tax ao inves de taxa) retorna código 404
err = get(h+'/tax?genus=serrasalmus&Species=spilopleura')
print('Código err:',err.status_code)
print(err.text)

# algo com termo errado (genus, ao invés de Genus) volta com dados errados. 
# ou seja é preciso ter cuidado
r2 = get(h+'taxa?genus=serrasalmus&Species=spilopleura')
print(r2.status_code)
print(r2.text,'\n')

# como exbido acima o conteúdo r.text é to tipo str,
# apesar de estar no formato JSON. 
# para ser útil teriamos de converter para JSON que por sua vez 
# se tornaria um dic do Python.
# para tanto existe a biblioteca json
import json
dados = json.loads(r.text)
print('Tipo Dado:',type(dados))
print(dados,'\n')

# por outro lado a função get pode nos facilitar, pois permite retornar 
# diretamente json sem usar a biblioteca 'json' e se der erro vem o texto com o erro. 
dic = get(h+'/taxa?Genus=serrasalmus&Species=spilopleura').json()
print('Tipo d:',type(dic))

# é claro passar parametros assim pode levar a erros e não ser muito prático.
# felizmente há outra forma de passar parâmetros
# lembra do tipo dicionário?

buscar = {
    'Genus' : 'serrasalmus',
    'Species' : ''
}

buscar['Species'] = 'spilopleura'

d = get(h+'/taxa', params = buscar).json()
print('\nBem mais pratico')
print(d)

# é claro como vimos no episódio 1 podemos acessar só o dado que nos interessa
# por exemplo o código da especie SpecCode, que esta data.
speccode = d['data'][0]['SpecCode']
print('SpecCode é:',speccode)
print('Total encontrado é:',d['count'])
# observou que o count é o nº de elementos em data
# 

Bom já aprendemos como acessar o repositório e como pegar dados dele.

Mas no exemplo pegamos apenas uma espécie. e se quisermos todos os gêneros? como lidar com isso?

vamos ao exemplo:

In [0]:
# aqui vamos repetir a definição de funções pois você pode estar começando daqui.
# mas se esta na linha do comando do Python3 ou do ipython, e fazendo em seguida não seria necessário. ou bastaria
# apertar a seta acima e acessar os comando anteriores
from requests import get
h = "https://fishbase.ropensci.org/"
pars = {
    'Genus' : 'serrasalmus',
    'Species' : ''
}

# observe que encontrou 'count' 31 mas retornou 'returned' 10
d = get(h+'taxa', params = pars).json()
print(d)

# por padrão o fishbase tem um limite 'limit' de 10 e começa pelo 'offset' 0 
# assim para pegar todos 
pars['limit'] = 31
d = get(h+'taxa', params = pars).json()
print('Total:',d['count'],', Recebido:',d['returned'],'\n')

# é claro carregar um volume muito grande de dados poderia encher a memória
# então vamos fazer isso usando ciclos e baixando de 12 em 12 por exemplo.
# e exibir os nomes das espécies e o SpecCode
# para isso vamos usar os parametros 'limit' e depois o 'offset'
# para ir buscando 
passo = 12
pars['limit'] = passo
r = get(h+'taxa', params = pars).json()
print('Total:',r['count'],', Recebido:',r['returned'],'\n')

total = r['count']
posicao = 0  # posição inicial 
print('--------------------------------------------')
while total > passo:
  # ***observe aqui****, que antes foi verificado que o campo 'data' 
  # é uma lista 'list' por isso é muito importante sempre olhar os dados 
  # para ver como eles são e assim saber como trabalhar com eles 
  especies = r['data']
  print('de:',posicao,' à ',posicao+passo)
  print('--------------------------------------------')
  for especie in especies:
    print('SpecCode:',especie['SpecCode'])
    print('Gênero:',especie['Genus'])
    print('Espécie:',especie['Species'],'\n')
  print('--------------------------------------------')
  posicao += passo
  pars['offset'] = posicao 
  total -= passo

print('Fim :)')

### **Tarefa 01**
No [Episódio 1](https://gist.github.com/jobdiogenes/a8d65fcabc8f8a47473769a9ba970fe0) você aprendeu como exibir todos os pares de dados de um dicionário. Use esse conhecimento e refaça a parte final do exemplo acima onde se exibe 'SpecCode', Genus e Species, para exibir todos os campos. A resposta você pode ver [aqui](https://github.com/jobdiogenes/data-science-recipes/blob/master/gists/code/answer-e01t01.py). Não veja isso agora. Faça esta tarefa. Mas uma dica você aprendeu isso ao final do episódio 01

***

## Fim do episódio 02
No próximo episódio iremos ver extrair dados integrando várias partes (rotas) dos dados oferecidos pelo repositório

> Você já encontrou repositórios que lhe interessam? 

> Se você ama taxonomia de espécies vai gostar de conhecer  o [Catalago of Life](http://www.catalogueoflife.org/) que é um repositório que aglutina informações oriundas de diversas fontes para todo reino de espécies da vida (na terra por enquanto).